# Courses and Teachers!

In [ ]:
# first, import the re module to use python's regular expressions
import re

In [ ]:
with open('datasets/ahs23-24/courses.txt', 'r') as fp:
    courses_upper = fp.read()
courses_upper

In [ ]:
# find a six-character run of capital/number surrounded by whitespace \s
pattern = "\s([A-Z0-9]{6})\s"
# find credits, course, and symbol
pattern = r"(1.0|0.5|0.25) ([\w\s\(\)\&\-\/]+(?:Req\.\))?) ([A-Z0-9]{5,6})"
courses = re.findall(pattern, courses_upper)
print(len(courses), "courses matched")
courses

In [ ]:
with open('datasets/ahs23-24/courses-frosh.txt', 'r') as fp:
    courses_frosh = fp.read()
courses_frosh

In [ ]:
# find a six-character run of capital/number surrounded by whitespace \s
pattern = "\s([A-Z0-9]{6})\s"
# find credits, course, and symbol
pattern = r"(1.0|0.5|0.25) ([\w\s\(\)\&\-\/]+(?:Req\.\))?) ([A-Z0-9]{5,6})"
courses = re.findall(pattern, courses_frosh)
print(len(courses), "courses matched")
courses

In [ ]:
with open('datasets/ahs23-24/skyward-courses.txt', 'r') as fp:
    data = fp.read()
data


In [ ]:
# find a six-character run of capital/number followed by a \t (tab character)
pattern = r"([A-Z0-9]{6})\t(.*)\n([\d\.]+) Credits,  (Elective|Required)"
# find symbol, course, and credits, and elective/required
courses = re.findall(pattern, data)
print(len(courses), "courses matched")
courses

In [ ]:
def parse_courses():
    courses = []
    with open('datasets/ahs23-24/skyward-courses.txt', "r") as file:
        coursedata = file.read()
    # group name syntax: https://docs.python.org/3/library/re.html#re.Match.group
    for item in re.finditer('(?P<key>[A-Z0-9]{6})\t(?P<course>.+)\n(?P<credits>[\d\.]+) Credits,  (?P<type>Elective|Required)', coursedata):
        foo = item.groupdict()
        foo['credits'] = float(foo['credits'])
        courses.append(foo)
    return courses

courses = parse_courses()
print(len(courses), "courses matched")
courses

In [ ]:
import re
import csv

def parse_filename_with_regex(filename, regex):
    courses = []
    with open(filename, "r") as file:
        coursedata = file.read()
    
    for item in regex.finditer(coursedata):
        course = item.groupdict()
        course['credits'] = float(course['credits'])
        courses.append(course)
    return courses

skyward_regex = re.compile(r"(?P<key>[A-Z0-9]{6})\t(?P<course>.+)\n(?P<credits>[\d\.]+) Credits,  (?P<type>Elective|Required)")
ahs_regex = re.compile(r"(?P<credits>1.0|0.5|0.25) (?P<course>[\w\s\(\)\&\-\/]+(?:Req\.\))?) (?P<key>[A-Z0-9]{5,6})")
skyware_file = 'datasets/ahs23-24/skyward-courses.txt'
courses_frosh_file = 'datasets/ahs23-24/courses-frosh.txt'
courses_file = 'datasets/ahs23-24/courses.txt'

all_courses = []
all_courses.extend(parse_filename_with_regex(skyware_file,skyward_regex))
all_courses.extend(parse_filename_with_regex(courses_file,ahs_regex))
all_courses.extend(parse_filename_with_regex(courses_frosh_file,ahs_regex))

outfile = 'datasets/courses.csv'
with open(outfile, 'w') as csvfile:
    fieldnames = ['key','course','credits','type']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for course in sorted(all_courses, key=lambda course: course['key']):
        writer.writerow(course)

print("wrote",len(all_courses),"courses to",outfile)

What can we discover with this data?
- How many teachers does the school need to run this course load?  How many periods do they teach?
- If we were going to schedule classes, what algorithm can we imagine?
- How do we de-dupe this data?